# This notebook is first used to predict fuel prices using fbprophet and then visualize the result using Tableau

1) Scrape data from BLS website : https://data.bls.gov/pdq/SurveyOutputServlet and clean the data

2) Use fb prophet to predict future prices

3) Save the result in an an excel file with a flag for actual or predicted value 

4) Connect to tableau and Visualize the predictions 

# Time Series modelling 

1) Assumptions : One assumption is that data is stationary, meaning that if you take different equal intervals the average or variance of the data is not changing 

In [6]:
import pandas as pd
from fbprophet import Prophet

import requests
import json
import prettytable

In [5]:
!pip install prettytable

In [20]:
# Scrape data from BLS website 

weburl= " https://data.bls.gov/pdq/SurveyOutputServlet" 
#Series ID from this URL needs to be changed to get different data 

headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['APU000074714'],"startyear":"1976", "endyear":"2020"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
for series in json_data['Results']['series']:
    x=prettytable.PrettyTable(["series id","year","period","value","footnotes"])
    seriesId = series['seriesID']
    for item in series['data']:
        year = item['year']
        period = item['period']
        value = item['value']
        footnotes=""
        for footnote in item['footnotes']:
            if footnote:
                footnotes = footnotes + footnote['text'] + ','
        if 'M01' <= period <= 'M12':
            x.add_row([seriesId,year,period,value,footnotes[0:-1]])
    output = open(seriesId + '.csv','w')
    output.write (x.get_string())
    output.close()

In [21]:
#This will be written to a file in memory called output.name
print("Output File name is",output.name)


Output File name is APU000074714.csv


In [25]:
fueldf = pd.read_csv(output.name , sep = "|",skiprows=1)
fueldf.drop(columns = ["Unnamed: 0","Unnamed: 6"],inplace= True)
fueldf

,series id,year,period,value,footnotes
0,NaN,NaN,NaN,NaN,NaN
1,APU000074714,1985.0,M12,1.208,
2,APU000074714,1985.0,M11,1.207,
3,APU000074714,1985.0,M10,1.204,
4,APU000074714,1985.0,M09,1.216,
...,...,...,...,...,...
117,APU000074714,1976.0,M04,0.592,
118,APU000074714,1976.0,M03,0.594,
119,APU000074714,1976.0,M02,0.600,
120,APU000074714,1976.0,M01,0.605,


In [29]:
#Clean white space in columns by clearing white spaces in string and re-assigning
newcollist =[]
for col in fueldf.columns :
    newcolname = col.replace(" ", "")
    newcollist.append(newcolname)
print("newcollist is ",newcollist)
fueldf.columns = newcollist
fueldf.columns 

newcollist is  ['seriesid', 'year', 'period', 'value', 'footnotes']


Index(['seriesid', 'year', 'period', 'value', 'footnotes'], dtype='object')

In [33]:
#Remove all rows with footnotes so that we get a clean dataframe 
fueldf.fillna(0,inplace=True)
footnotes_nonzeromask = fueldf.footnotes != 0
fueldf_nofootnote = fueldf.loc[footnotes_nonzeromask,:]
fueldf_nofootnote

,seriesid,year,period,value,footnotes
1,APU000074714,1985.0,M12,1.208,
2,APU000074714,1985.0,M11,1.207,
3,APU000074714,1985.0,M10,1.204,
4,APU000074714,1985.0,M09,1.216,
5,APU000074714,1985.0,M08,1.229,
...,...,...,...,...,...
116,APU000074714,1976.0,M05,0.600,
117,APU000074714,1976.0,M04,0.592,
118,APU000074714,1976.0,M03,0.594,
119,APU000074714,1976.0,M02,0.600,


In [35]:
fueldf_nofootnote['footnotes'].describe()

count             120
unique              1
top                  
freq              120
Name: footnotes, dtype: object

In [38]:
fueldf_nofootnote['ActualorPredicted'] = 'Actual'
fueldf_nofootnote

C:\Users\Phanindra\Anaconda3\envs\timeseries\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,seriesid,year,period,value,footnotes,ActualorPredicted
1,APU000074714,1985.0,M12,1.208,,Actual
2,APU000074714,1985.0,M11,1.207,,Actual
3,APU000074714,1985.0,M10,1.204,,Actual
4,APU000074714,1985.0,M09,1.216,,Actual
5,APU000074714,1985.0,M08,1.229,,Actual
...,...,...,...,...,...,...
116,APU000074714,1976.0,M05,0.600,,Actual
117,APU000074714,1976.0,M04,0.592,,Actual
118,APU000074714,1976.0,M03,0.594,,Actual
119,APU000074714,1976.0,M02,0.600,,Actual


# 2) Use fb prophet to predict future prices 

fbprophet Needs data in two columns namely "DS" (date column) and "y" the actual column or here the price of fuel 

In [45]:
#Using lambda functions to extract Month number from period
fueldf_nofootnote.loc[:,'Month'] = fueldf_nofootnote.loc[:,'period'].apply(lambda row : row.replace("M",""))
fueldf_nofootnote.loc[:,'Month'] = fueldf_nofootnote.loc[:,'Month'] .astype(int)
fueldf_nofootnote

C:\Users\Phanindra\Anaconda3\envs\timeseries\lib\site-packages\pandas\core\indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


,seriesid,year,period,value,footnotes,ActualorPredicted,Month
1,APU000074714,1985.0,M12,1.208,,Actual,12
2,APU000074714,1985.0,M11,1.207,,Actual,11
3,APU000074714,1985.0,M10,1.204,,Actual,10
4,APU000074714,1985.0,M09,1.216,,Actual,9
5,APU000074714,1985.0,M08,1.229,,Actual,8
...,...,...,...,...,...,...,...
116,APU000074714,1976.0,M05,0.600,,Actual,5
117,APU000074714,1976.0,M04,0.592,,Actual,4
118,APU000074714,1976.0,M03,0.594,,Actual,3
119,APU000074714,1976.0,M02,0.600,,Actual,2


In [46]:
fueldf_nofootnote['DS'] = pd.to_datetime(dict(year=fueldf_nofootnote.year , month=fueldf_nofootnote.Month, day= 1))
fueldf_nofootnote

C:\Users\Phanindra\Anaconda3\envs\timeseries\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,seriesid,year,period,value,footnotes,ActualorPredicted,Month,DS
1,APU000074714,1985.0,M12,1.208,,Actual,12,1985-12-01
2,APU000074714,1985.0,M11,1.207,,Actual,11,1985-11-01
3,APU000074714,1985.0,M10,1.204,,Actual,10,1985-10-01
4,APU000074714,1985.0,M09,1.216,,Actual,9,1985-09-01
5,APU000074714,1985.0,M08,1.229,,Actual,8,1985-08-01
...,...,...,...,...,...,...,...,...
116,APU000074714,1976.0,M05,0.600,,Actual,5,1976-05-01
117,APU000074714,1976.0,M04,0.592,,Actual,4,1976-04-01
118,APU000074714,1976.0,M03,0.594,,Actual,3,1976-03-01
119,APU000074714,1976.0,M02,0.600,,Actual,2,1976-02-01


In [47]:
fuel_fbp_input = fueldf_nofootnote.loc[:, ['DS','value']]
fuel_fbp_input

,DS,value
1,1985-12-01,1.208
2,1985-11-01,1.207
3,1985-10-01,1.204
4,1985-09-01,1.216
5,1985-08-01,1.229
...,...,...
116,1976-05-01,0.600
117,1976-04-01,0.592
118,1976-03-01,0.594
119,1976-02-01,0.600
